In [1]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [3]:
import collect_stats as cs

In [336]:
exp_folder = '/cluster/sj1/bb_opt/experiments'
test_filepath = '/cluster/sj1/bb_opt/data/tatsu_pbm/test_files'
filedir = '/cluster/sj1/bb_opt/data/paper_data2'
num_tfs_to_test = 38
batches = [10, 20, 100, 200]
batches = [10]
num_acks = 15

In [5]:
with open(test_filepath, 'r') as f:
    filenames = [k.strip() for k in f.readlines()][:num_tfs_to_test]

In [6]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

In [7]:
filenames = [
    'HESX1_E149K_R1_8mers.txt',
    'EGR2_D383Y_R1_8mers.txt',
    'BCL6_H676Y_R1_8mers.txt',
    'ARX_L343Q_R1_8mers.txt',
    'CRX_E80A_R1_8mers.txt',
    'ESX1_K193R_R1_8mers.txt',
    'FOXC1_F112S_R1_8mers.txt',
    'GFI1B_A204T_R1_8mers.txt',
    'GFI1_L400F_R1_8mers.txt',
    'HOXC4_N178S_R1_8mers.txt',
    'HOXD13_I322L_R1_8mers.txt',
    'ISX_R83Q_R1_8mers.txt',
    'KLF11_R402Q_R1_8mers.txt',
    'KLF1_E325K_R1_8mers.txt',
    'MSX2_P148H_R1_8mers.txt',
    'NKX2-5_K183E_R1_8mers.txt',
    'NR1H4_C144R_R1_8mers.txt',
    'NR2E3_G56R_R1_8mers.txt',
    'OVOL2_D228E_R1_8mers.txt',
    'PAX3_G48R_R1_8mers.txt',
    'PAX4_R183C_R1_8mers.txt',
    'PAX6_A79E_R1_8mers.txt',
    'PAX7_P112L_R1_8mers.txt',
    'PBX4_R215Q_R1_8mers.txt',
    'PHOX2B_Q143R_R1_8mers.txt',
    'PITX2_L100Q_R1_8mers.txt',
    'POU3F4_A237G_R1_8mers.txt',
    'POU4F3_K277R_R1_8mers.txt',
    'POU6F2_E639K_R1_8mers.txt',
    'PROP1_R112Q_R1_8mers.txt',
    'SIX6_H141N_R1_8mers.txt',
    'SNAI2_D119E_R1_8mers.txt',
    'VAX2_L139M_R1_8mers.txt',
    'VENTX_E101K_R1_8mers.txt',
    'VSX1_G160D_R1_8mers.txt',
    'WT1_F392L_R1_8mers.txt',
    'ZNF200_H322Y_R1_8mers.txt',
    'ZNF655_E327G_R1_8mers.txt',
]

In [8]:
tf_max = {}
tf_labels = {}
for tf in filenames:
    labels = np.load(filedir + "/" + tf + "/labels.npy")
    labels = np.log(labels)
    tf_max[tf] = float(labels.max())
    tf_labels[tf] = labels

In [9]:
arrs = {}

In [10]:
import collect_stats as cs

In [340]:
to_read = {
    #'ensemble7/o_none_ucb_modelseed_': [None, 'e7'],
    #'ensemble7/o_none_ucb_ucb_step_0.1_modelseed_': [None, 'e7_ucb_step_0.1'],
    #'ensemble7/o_none_ucb_maxvar_g000510204080_modelseed_': [None, 'e7_maxvar_g80'],
    #'ensemble7/o_none_ucb_maxinoutvar_g000510204080_modelseed_': [None, 'e7_inoutvar_g80'],
    #'ensemble7/o_none_ucb_maxinvar_g000510204080_modelseed_': [None, 'e7_invar_g80'],
    #'ensemble7/o_none_ucb_dataseed_modelseed_': [None, 'e7'],
    #'ensemble7/o_none_ucb_invar_g000510204080_dataseed_modelseed_' : [None, 'e7_datarand_invar_g80'],
    #'ensemble7/o_none_ucb_maxvar_g000510204080_dataseed_modelseed_' : [None, 'e7_maxvar_g80'],
    #'ensemble7/o_none_ucb_inoutvar_g000510204080_dataseed_modelseed_': [None, 'e7_datarand_inoutvar_g80'],
    #'ensemble7/o_none_ucb_maxvar_inverse_g000510204080_': [None, 'MODD'],
    #'ensemble9/o_none_ucb_modelseed_': [None, 'e9'],
    #'ensemble9/o_none_ucb_maxvar_g000510204080_modelseed_': [None, 'e9_maxvar_g80'],
    #'ensemble9/o_none_ucb_maxinvar_g000510204080_modelseed_': [None, 'e9_invar_g80'],
    #'ensemble9/o_none_ucb_maxinoutvar_g000510204080_modelseed_': [None, 'e9_inoutvar_g80'],
    #'ensemble7/o_kriging_believer_kb_modelseed_': [None, 'e7_kb'],
    #'ensemble7/o_empirical_kb_div_3_modelseed_': [None, 'e7_empirical_kb'],
    #'ensemble7/o_empirical_kb_ucb_weighted_div_10_modelseed_': [None, 'e7_wekb_div10'],
    #'ensemble7/o_empirical_kb_ucb_weighted_0.5_div_10_modelseed_': [None, 'e7_wekb_0.5_div10'],
    #'ensemble7/o_empirical_kb_ucb_weighted_mean_0.5_div_10_modelseed_': [None, 'e7_wekb_mean_0.5_div10'],
    #'ensemble7/o_kb_dataseed_modelseed_': [None, 'e7_datarand_kb'],
    #'ensemble10/o_kb_dataseed_modelseed_': [None, 'e10_datarand_kb'],
    #'ensemble10/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_': [None, 'e10_datarand_wekb_div10'],
    #'ensemble11/o_kb_dataseed_modelseed_': [None, 'e11_datarand_kb'],
    #'ensemble11/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_': [None, 'e11_datarand_wekb_div10'],
    #'ensemble11/o_kb_malaria_dataseed_modelseed_': [None, 'mal_datarand_kb'],
    #'ensemble11/o_empirical_kb_malaria_ucb_weighted_0.5_div_10_dataseed_modelseed_': [None, 'mal_datarand_wekb_div10'],
    #'ensemble11/o_kb_nolog_malaria_dataseed_modelseed_': [None, 'mal_datarand_kb_nolog'],
    #'ensemble12/o_none_ucb_': [None, 'e7_2'],
    #'ensemble12/o_none_ucb_maxinvar_g000510204080_': [None, 'maxinvar2'],
    #'ensemble12/o_none_ucb_maxvar_g000510204080_': [None, 'maxvar2'],
    #'ensemble12/o_none_ucb_maxvar_id_g000510204080_': [None, 'modr2'],
    #'ensemble13/o_info_mves_div_5_minhsic_1_': [None, 'mves_div_5_minhsic_1'],
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0_': [None, 'mves_bs_0'],
    #'ensemble13/o_none_ucb_': [None, 'e13'],
    #'ensemble13/o_pdts_ucb_rand_randdiv_3_': [None, 'pdts_randdiv_3'],
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_': [None, 'mves_bs_0_it20'],
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_fill_pdts_': [None, 'mves_bs_0_it20_fp'],
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.0_bs_0_fill_pdts_': [None, 'mves_bs_0_it20_fp_mh_0'],
    #'ensemble13_init_train_20/o_none_ucb_': [None, 'e13_it20'],
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_3_': [None, 'e13_it20_randdiv_3'],
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_2_': [None, 'e13_it20_randdiv_2'],
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_1_': [None, 'e13_it20_randdiv_1'],
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_randdiv_1_': [None, 'mves_it20_fucb_randdiv_1'],
    #'ensemble13_init_train_20/o_ei_ucb_': [None, 'ei_it20'],
    #'ensemble13_init_train_20/o_pdts_ucb_rand_div_5_': [None, 'pdts_rand_div_5'],
    #'ensemble13_init_train_20/o_pdts_ucb_rand_div_3_': [None, 'pdts_rand_div_3'],
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_fucb_randdiv_3_': [None, 'mves_it20_div_5_randdiv_3_fucb'],
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_fp_randdiv_3_': [None, 'mves_it20_div_5_randdiv_3_fp'],
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0.9_': [None, 'mves_bs_0.9_it20'],
    #'ensemble13_init_train_20/o_pdts_ucb_': [None, 'pdts_it20'],
    #'ensemble13_init_train_20/o_info_mves_div_1_minhsic_0.05_bs_0_fill_pdts_': [None, 'mves_div_1_bs_0_it20'],
    #'ensemble13_init_train_20/o_er_pdts_ucb_div_5_': [None, 'er_pdts_div_5_it20'],
    #'ensemble13_init_train_10/o_info_mves_div_5_minhsic_0.05_bs_0_fill_pdts_': [None, 'mves_bs_0_it10_fp'],
    #'ensemble13_init_train_10/o_pdts_ucb_': [None, 'pdts_it10'],
    #'ensemble13/o_pdts_ucb_': [None, 'pdts'],
    'ensemble13/o_info_pdts_cond_div_2_fp_': [None, 'pdts_cond_fp'],
    #'ensemble13/o_info_mves_div_5_fp_': [None, 'mves_fp'],
    #'ensemble13/o_pdts_ucb_rand_rdiv_1_': [None, 'pdts_rand_rdiv_1'],
    #'ensemble13/o_none_ucb_rand_rdiv_1_': [None, 'ucb_rand_rdiv_1'],
    #'ensemble13/o_none_ucb_0.9_': [None, 'bs_0.9'],
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0.9_': [None, 'mves_bs_0.9'],
}

In [369]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=20, mode="bayes_opt")
    #cs.get_data(exp_folder, experiment, batches, num_samples=20, mode="bayes_opt")

reading ensemble13/o_info_pdts_cond_div_2_fp_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20


In [279]:
to_eval = [
    #'ensemble7/o_none_ucb_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_g000510204080_modelseed_',
    #'ensemble7/o_none_ucb_maxinvar_g000510204080_modelseed_',
    #'ensemble7/o_none_ucb_maxinoutvar_g000510204080_modelseed_',
    #'ensemble7/o_none_ucb_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_invar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_inoutvar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_inverse_g000510204080_',
    #'ensemble7/o_kriging_believer_kb_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_0.5_div_10_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_mean_0.5_div_10_modelseed_',
    #'ensemble7/o_kb_dataseed_modelseed_',
    #'ensemble10/o_kb_dataseed_modelseed_',
    #'ensemble10/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    #'ensemble11/o_kb_dataseed_modelseed_',
    #'ensemble11/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    #'ensemble11/o_kb_malaria_dataseed_modelseed_',
    #'ensemble11/o_empirical_kb_malaria_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    #'ensemble12/o_none_ucb_',
    #'ensemble12/o_none_ucb_maxinvar_g000510204080_',
    #'ensemble12/o_none_ucb_maxvar_g000510204080_',
    #'ensemble12/o_none_ucb_maxvar_id_g000510204080_',
    'ensemble13/o_none_ucb_',
    'ensemble13/o_pdts_ucb_',
    'ensemble13/o_info_pdts_cond_div_2_fp_',
    'ensemble13/o_info_mves_div_5_fp_',
    'ensemble13/o_pdts_ucb_rand_rdiv_1_',
    'ensemble13/o_none_ucb_rand_rdiv_1_',
    #'ensemble13/o_info_mves_div_5_minhsic_1_',
    'ensemble13/o_pdts_ucb_rand_randdiv_3_',
    'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0_',
    
    #'ensemble13_init_train_20/o_none_ucb_',
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_1_',
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_2_',
    #'ensemble13_init_train_20/o_none_ucb_rand_randdiv_3_',
    #'ensemble13_init_train_20/o_ei_ucb_',
    #'ensemble13_init_train_20/o_pdts_ucb_',
    #'ensemble13_init_train_20/o_pdts_ucb_rand_div_5_',
    #'ensemble13_init_train_20/o_pdts_ucb_rand_div_3_',
    #'ensemble13_init_train_20/o_er_pdts_ucb_div_5_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0.9_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_fill_pdts_',
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_fp_randdiv_3_',
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_fucb_randdiv_3_',
    #'ensemble13_init_train_20/o_info_rand_mves_div_5_randdiv_1_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.0_bs_0_fill_pdts_',
    #'ensemble13_init_train_20/o_info_mves_div_1_minhsic_0.05_bs_0_fill_pdts_',
    #'ensemble13_init_train_10/o_info_mves_div_5_minhsic_0.05_bs_0_fill_pdts_',
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0.9_',
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0.95_',
    #'ensemble13/o_none_ucb_0.9_',
]

In [15]:
idx_data_extractor = lambda x, filename : x['idx_frac'][0].item()

In [16]:
ack_rel_opt_val_data_extractor = lambda x, filename : x['ack_rel_opt_value']

In [17]:
train_rmse_fn = lambda x, filename : x['logging'][1]['train']['std']

In [18]:
def ir_regret_fn(x, filename): 
    temp = x['ir_batch_cur_idx'].numpy().tolist()
    temp2 = set(x['ack_idx'].numpy().tolist())
    temp3 = None
    for k in temp[::-1]:
        if k not in temp2:
            break
    return np.exp(max(tf_labels[filename][k], tf_labels[filename][x['ack_idx']].max()))

def ack_regret_fn(x, filename): 
    return np.exp(max(tf_labels[filename][x['ack_idx']].max(), tf_labels[filename][x['idx_at_each_iter'][0]].max()))
    #return np.exp(tf_labels[filename][x['ack_idx']].max())



In [364]:
cs.prop_test(20, 
             filenames,
             idx_data_extractor,
             #ack_rel_opt_val_data_extractor, 
             arrs, 
             [to_eval[1], to_eval[2]],
             14, 
             pval_threshold=.5,
             paired_test=False,
            )

comparing pdts pdts_cond_fp
combined pval: nan vs nan
count: 0/0


In [374]:
cs.prop_test(10, 
             filenames, 
             ack_regret_fn,
             #ack_rel_opt_val_data_extractor,
             arrs,
             [to_eval[1], to_eval[2]],
             14,
             pval_threshold=.2,
             paired_test=True,
            )

comparing pdts pdts_cond_fp
CRX_E80A_R1_8mers.txt 0.18467542651991772 256524.2 273831.28 1 	 (std: 45040.78906 37841.54297, #n: 20 20)
OVOL2_D228E_R1_8mers.txt 0.15967378238118082 122091.36 112546.086 0 	 (std: 27837.40625 6562.03613, #n: 20 20)
SIX6_H141N_R1_8mers.txt 0.1175479368230219 122484.625 115578.73 0 	 (std: 18219.81250 4739.90869, #n: 20 20)
combined pval: 0.09339 vs 0.18468
count: 1/3


In [ ]:
cs.prop_test(20, 
             filenames, 
             ir_regret_fn, 
             arrs, 
             [to_eval[3], to_eval[5]], 
             14,
             pval_threshold=.5,
             paired_test=True,
            )

In [ ]:
cs.uniform_baseline(
    3,
    tf_labels,
    20, 
    filenames, 
    ack_regret_fn, 
    #ack_rel_opt_val_data_extractor,
    arrs,
    to_eval[5], 
    14, 
    pval_threshold=1, 
)

In [ ]:
cs.prop_test(100, 
             filenames, 
             idx_data_extractor, 
             arrs, 
             [to_eval[-2], to_eval[-1]], 
             9,
             pval_threshold=0.2,
             paired_test=True,
            )

In [ ]:
cs.prop_test(100, filenames, ir_regret_fn, arrs, [to_eval[-2], to_eval[-1]], 9, pval_threshold=0.2, paired_test=True)

In [ ]:
cs.prop_test(100, filenames, ack_regret_fn, arrs, [to_eval[-2], to_eval[-1]], 9, pval_threshold=1.4, paired_test=True)

In [ ]:
cs.prop_test(200, 
             filenames, 
             idx_data_extractor, 
             arrs, 
             [to_eval[-2], to_eval[-1]], 
             29, 
             pval_threshold=1,
             paired_test=True,
            )

In [ ]:
cs.prop_test(200, filenames, ack_rel_opt_val_data_extractor, arrs, [to_eval[-2], to_eval[-1]], 29, pval_threshold=1.4, paired_test=True)

In [225]:
to_eval = [
    #'ensemble7/o_none_ucb_modelseed_',
    #'ensemble7/o_none_ucb_ucb_step_0.1_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_g000510204080_modelseed_',
    #'ensemble7/o_none_ucb_maxinoutvar_g000510204080_modelseed_',
    #'ensemble7/o_none_ucb_maxinvar_g000510204080_modelseed_',
    
    #'ensemble7/o_none_ucb_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_invar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_none_ucb_maxvar_inverse_g000510204080_',
    
    #'ensemble7/o_none_ucb_inoutvar_g000510204080_dataseed_modelseed_',
    #'ensemble7/o_kriging_believer_kb_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_0.5_div_10_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_mean_0.5_div_10_modelseed_',
    #'ensemble7/o_empirical_kb_div_3_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_div_10_modelseed_',
    #'ensemble7/o_kb_dataseed_modelseed_',
    #'ensemble7/o_empirical_kb_ucb_weighted_div_10_dataseed_modelseed_',
    #'ensemble10/o_kb_dataseed_modelseed_',
    #'ensemble10/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    #'ensemble11/o_kb_dataseed_modelseed_',
    #'ensemble11/o_empirical_kb_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    #'ensemble11/o_kb_malaria_dataseed_modelseed_',
    #'ensemble11/o_empirical_kb_malaria_ucb_weighted_0.5_div_10_dataseed_modelseed_',
    
    #'ensemble12/o_none_ucb_',
    #'ensemble13/o_info_mves_div_5_minhsic_1_',
    'ensemble13/o_none_ucb_',
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0.95_',
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0.9_',
    #'ensemble13/o_info_mves_div_5_minhsic_0.05_bs_0_',
    #'ensemble13/o_none_ucb_0.9_',
    'ensemble13/o_pdts_ucb_',
    #'ensemble13_init_train_20/o_none_ucb_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0_fill_pdts_',
    #'ensemble13_init_train_20/o_info_mves_div_5_minhsic_0.05_bs_0.9_',
    #'ensemble13_init_train_20/o_pdts_ucb_',
]

In [349]:
ack_iter = 14
for batch_size in [10]:
    print("BATCH SIZE", batch_size)
    for filename in filenames:
        filename_printed = False
        for experiment in to_eval:
            if len(arrs[experiment][0]) == 0:
                continue
            idx_frac = None
            num_used = 0
            val_nll = 0
            try:
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) <= ack_iter:
                        continue
                    num_used += 1
                    mean = 0
                    #for i in range(len(stats[filename][batch_size])):
                        #mean += stats[filename][batch_size][i]['logging'][1]['best']['nll']
                    if len(stats[filename][batch_size]) > 0:
                        mean /= (len(stats[filename][batch_size]))
                    #val_nll += stats[filename][batch_size][0]['logging'][1]['best']['nll']
                    val_nll += mean
                    if idx_frac is None:
                        idx_frac = stats[filename][batch_size][ack_iter]['idx_frac']
                        #idx_frac = tf_labels[filename][stats[filename][batch_size][ack_iter]['ir_batch_cur_idx'][-1]]
                        #idx_frac = stats[filename][batch_size][ack_iter]['corr_stats'][1]
                    else:
                        idx_frac = idx_frac + stats[filename][batch_size][ack_iter]['idx_frac']
                        #idx_frac = torch.max(idx_frac, stats[filename][batch_size][ack_iter]['idx_frac'])
                        #idx_frac = idx_frac + tf_labels[filename][stats[filename][batch_size][ack_iter]['ir_batch_cur_idx'][-1]]
                        #idx_frac = idx_frac + stats[filename][batch_size][ack_iter]['corr_stats'][1]
                if num_used > 0:
                    if not filename_printed:
                        print(filename)
                        filename_printed = True
                    idx_frac = idx_frac/num_used
                    #idx_frac = idx_frac.numpy()
                    print(arrs[experiment][1] + ":", " ".join(("{:6.6f}".format(k) for k in idx_frac)), ";", "{:6.2f}".format(val_nll/num_used), "val_nll ;", str(num_used), "samples")
                    #print(arrs[experiment][1] + ":", str(idx_frac), ";", "{:6.2f}".format(val_nll/num_used), "val_nll ;", str(num_used), "samples")
            except Exception as e:
                pass

BATCH SIZE 10
HESX1_E149K_R1_8mers.txt
e13: 0.160671 0.067275 0.037853 0.020185 ;   0.00 val_nll ; 10 samples
pdts: 0.178049 0.068187 0.037747 0.020155 ;   0.00 val_nll ; 20 samples
EGR2_D383Y_R1_8mers.txt
e13: 0.009756 0.006934 0.008179 0.009561 ;   0.00 val_nll ; 10 samples
pdts: 0.021799 0.011861 0.010292 0.009804 ;   0.00 val_nll ; 20 samples
BCL6_H676Y_R1_8mers.txt
e13: 0.212500 0.076825 0.041502 0.021037 ;   0.00 val_nll ; 10 samples
pdts: 0.208689 0.077281 0.041821 0.021113 ;   0.00 val_nll ; 20 samples
ARX_L343Q_R1_8mers.txt
e13: 0.077439 0.041119 0.026604 0.016644 ;   0.00 val_nll ; 10 samples
pdts: 0.082774 0.041819 0.028109 0.017085 ;   0.00 val_nll ; 20 samples
CRX_E80A_R1_8mers.txt
e13: 0.045122 0.051582 0.034722 0.020049 ;   0.00 val_nll ; 10 samples
pdts: 0.048628 0.052190 0.035041 0.020216 ;   0.00 val_nll ; 20 samples
pdts_cond_fp: 0.045732 0.051247 0.034372 0.020018 ;   0.00 val_nll ; 20 samples
ESX1_K193R_R1_8mers.txt
e13: 0.213720 0.072567 0.040164 0.021082 ;   0.00

In [ ]:
import collect_stats as cs

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'relative optimal value',
    #ack_regret_fn,
    ack_rel_opt_val_data_extractor,
    14,
    arrs,
    to_eval,
    legend_loc=3,
    figsize=(6, 4),
    num_samples_label=True,
    save_path='/cluster/sj1/bb_opt/plots',
)

In [ ]:
cs.plot_data_vs_ack_iter(
    10,
    filenames[0:15],
    'avg_seeds',
    'gamma',
    lambda x : x['best_gamma'] if x['best_gamma'] is not None else 0,
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
ack_iter = 0
for filename in filenames:
    print(filename)
    for batch_size in batches:
        for experiment in to_eval:
            best_value = 0
            num_used = 0
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                num_used += 1
                best_value += stats[filename][batch_size][ack_iter]['logging'][9].item()
            if num_used > 0:
                best_value = best_value/num_used
                print(arrs[experiment][1] + ":", best_value, ";", str(num_used), "samples")

In [ ]:
test_properties = [
    'test_log_prob',
    'test_mse',
    'test_kt_corr',
    'test_std_list',
    'test_mse_std_corr'
]

avg = True

for prop in test_properties[1:2]:
    for filename in filenames[0:1]:
        print(filename)
        for batch_size in batches:
            for experiment in arrs:
                plt.figure(figsize=(15, 4))
                plt.subplot(121)
                prop_list = np.array([0.]*num_acks)
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        prop_list += np.array([stats[filename][batch_size][i][prop][0].item() for i in range(num_acks)])
                    else:
                        prop_list = [stats[filename][batch_size][i][prop][0].item() for i in range(num_acks)]
                    if not avg:
                        plt.plot(prop_list)
                if avg:
                    plt.plot(prop_list/num_used)
                plt.title(arrs[experiment][1] + "; " + prop + "; all")
                plt.subplot(122)
                prop_list = np.array([0.]*num_acks)
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        prop_list += np.array([stats[filename][batch_size][i][prop][1].item() for i in range(num_acks)])
                    else:
                        prop_list = [stats[filename][batch_size][i][prop][1].item() for i in range(num_acks)]
                    if not avg:
                        plt.plot(prop_list)
                if avg:
                    plt.plot(prop_list/num_used)
                plt.title(arrs[experiment][1] + "; " + prop + "; top 10%")

In [ ]:
ack_iter = 0
for filename in filenames[2:3]:
    print(filename)
    for batch_size in batches:
        for experiment in arrs:
            plt.figure(figsize=(15, 4))
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                prop_list = stats[filename][batch_size][ack_iter]['logging'][-1].numpy()
                plt.plot(prop_list)
                plt.title(arrs[experiment][1] + "; " + str(ack_iter))

In [ ]:
to_eval = [
    #'top_val_none_ucb_g0.0_',
    'top_val_none_ucb_maxstd_g01051020_',
    'top_val_none_ucb_g0.0_200epochs_last_',
    #'top_val_none_ucb_maxvar_g01051020_',
]

In [ ]:
ack_iter = 1
for filename in filenames[:3]:
    print(filename)
    for batch_size in batches:
        for experiment in to_eval:
            plt.figure(figsize=(15, 4))
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                #prop_list = stats[filename][batch_size][ack_iter]['logging'][5].numpy()
                prop_list = stats[filename][batch_size][ack_iter]['best_gamma']
                plt.plot(prop_list)
                plt.title(arrs[experiment][1] + "; " + str(ack_iter))

In [ ]:
test_properties = [
    'test_log_prob',
    'test_mse',
    'test_kt_corr',
    'test_std_list',
    'test_mse_std_corr'
]

ack_iter = -1
for filename in filenames:
    print(filename)
    for batch_size in [10]:
        for experiment in to_eval:
            m = None
            num_used = 0
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                num_used += 1
                #m += stats[filename][batch_size][ack_iter]['test_std_list'][0].item()
                m2 = np.array([
                    np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][0].item(),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][1].item(),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][2].item()
                ])
                if m is None:
                    m = m2
                else:
                    m += m2
            if num_used > 0:
                print(arrs[experiment][1], ":", m/num_used, ";", num_used, "samples")

In [ ]:
num_labels = 32897

In [ ]:
to_eval = [
    'take_log_none_ucb_c1.0_g0.0_',
    #'take_log_none_ucb_c2.0_g0.0_',
    'take_log_none_ucb_c1.0_g5.0_',
    'take_log_none_ucb_c1.0_g10.0_',
    'take_log_none_ucb_c1.0_g20.0_',
]

to_eval = [
    'take_log_none_ucb_num_acks_30_c1.0_g0.0_',
    'take_log_none_ucb_num_acks_30_c1.0_g5.0_',
    'take_log_none_ucb_num_acks_30_c1.0_g10.0_',
]

In [ ]:
num_acks = 30

In [ ]:
to_eval = {
    'take_log_none_ucb_num_acks_30_c1.0_g0.0_': 'Normal',
    'take_log_none_ucb_num_acks_30_c1.0_maxvar_' : 'MOD',
    'take_log_none_ucb_num_acks_30_c1.0_mincorr_' : 'MOD2',
}

to_eval = {
    'take_log_none_ucb_ntest' : 'Normal',
}

In [ ]:
to_eval = [
    #'top_val_none_ucb_g0.0_',
    'top_val_none_ucb_g0.0_modelseed',
    #'top_val_none_ucb_maxstd_g01051020_',
    #'top_val_none_ucb_g0.0_200epochs_last_',
    #'top_val_none_ucb_maxvar_g01051020_',
]

In [ ]:
avg = True
num_acks = 10

for batch_size in batches:
    for filename in filenames:
        if avg:
            plt.figure(figsize=(6, 4))
            legend = []
        for experiment in to_eval:
            if not avg:
                plt.figure(figsize=(15, 4))
            prop_list = []
            num_used = 0
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                if len(stats[filename][batch_size]) < num_acks:
                    continue
                num_used += 1
                if avg:
                    #prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                    #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
                    #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][0][-4]) for ack_iter in range(num_acks)])]
                    prop_list += [np.array([(stats[filename][batch_size][ack_iter]['logging'][0][8]) for ack_iter in range(num_acks)])]
                else:
                    prop_list = [(stats[filename][batch_size][ack_iter]['ack_labels'].max()) for ack_iter in range(num_acks)]
                    prop_list = [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][0][-4]) for ack_iter in range(num_acks)])]
                if not avg:
                    plt.plot(prop_list)
            if avg and num_used > 0:
                prop_list = np.stack(prop_list, axis=0)
                plt.plot(np.median(prop_list, axis=0))
                legend += [arrs[experiment][1]]
            elif not avg:
                plt.title(arrs[experiment][1] + "; " + str(batch_size) + "; all")
        if avg:
            plt.legend(legend)
            plt.xlabel('Acquisition Iteration')
            plt.ylabel('Relative optimal value')
            #plt.title(filename + "; " + str(batch_size) + "; all")
            plt.title(filename.split("_")[0])

In [ ]:
avg = True

for batch_size in batches:
    for filename in filenames:
        if avg:
            plt.figure(figsize=(6, 4))
            legend = []
        for experiment in to_eval:
            if not avg:
                plt.figure(figsize=(15, 4))
                
            for prop_i in [5, 6]:
                prop_list = []
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        #prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                        #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
                        #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][1][prop_i].item()) for ack_iter in range(num_acks)])]
                        prop_list += [np.array([(stats[filename][batch_size][ack_iter]['best_gamma']) for ack_iter in range(num_acks)])]
                    else:
                        prop_list = [(stats[filename][batch_size][ack_iter]['ack_labels'].max()) for ack_iter in range(num_acks)]
                        prop_list = [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][1][prop_i].item()) for ack_iter in range(num_acks)])]
                    if not avg:
                        plt.plot(prop_list)
                if avg and num_used > 0:
                    prop_list = np.stack(prop_list, axis=0)
                    plt.plot(np.median(prop_list, axis=0))
                    legend += [str(prop_i)]
                elif not avg:
                    plt.title(arrs[experiment][1] + "; " + str(batch_size) + "; all")
        if avg:
            plt.legend(legend)
            plt.xlabel('Acquisition Iteration')
            plt.ylabel('Relative optimal value')
            #plt.title(filename + "; " + str(batch_size) + "; all")
            plt.title(filename.split("_")[0])

In [ ]:
avg = True

plt.figure(figsize=(6, 4))
legend = []
for experiment in to_eval:
    prop_list = []
    for batch_size in [10]:
        for filename in filenames:
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                if len(stats[filename][batch_size]) < num_acks:
                    continue
                prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
    prop_list = np.stack(prop_list, axis=0)
    plt.plot(np.median(prop_list, axis=0))
    #legend += [arrs[experiment][1]]
    legend += [to_eval[experiment]]
plt.legend(legend)
plt.xlabel('Acquisition Iteration')
plt.ylabel('Relative optimal value')